In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os 
from warnings import simplefilter
import pandas as pd
from imblearn.over_sampling import SMOTE  

In [2]:
import models
import class_sampling
import train
import metric_utils
import inference
import loss_fns
import torchvision.ops 

In [3]:
NUM_CLASSES = 10
n_epochs = 30
batch_size_train = 64
batch_size_test = 1000
momentum = 0

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

NUM_CLASSES_REDUCED = 2
nums = (0, 1)
ratio = (100, 1)

CLASS_LABELS = {'airplane': 0,
                 'automobile': 1,
                 'bird': 2,
                 'cat': 3,
                 'deer': 4,
                 'dog': 5,
                 'frog': 6,
                 'horse': 7,
                 'ship': 8,
                 'truck': 9}


simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)
simplefilter(action='ignore', category=DeprecationWarning)

In [4]:
col_names = ["name", 
            "num_classes", 
            "classes_used", 
            "ratio", 
            "learning_rate", 
            "mean_0", "variance_0",
            "mean_10", "variance_10",
            "mean_20", "variance_20",
            "mean_30", "variance_30",
          #   "mean_40", "variance_40",
          #   "mean_50", "variance_50",
             "cap", "normalization"]

rows = []

In [5]:
norm=True

if norm:
    transform=torchvision.transforms.Compose([torchvision.transforms.Normalize(mean=[143.8888, 127.1705, 117.5357], std=[69.8313, 64.5137, 66.9933])])
else:
   # transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
    transform=None

train_CIFAR10 = torchvision.datasets.CIFAR10('cifar10', train=True, download=True,
                             transform=transform)  


test_CIFAR10 = torchvision.datasets.CIFAR10('cifar10', train=False, download=True,
                             transform=transform)  

train_CIFAR10.data = train_CIFAR10.data.reshape(50000, 3, 32, 32)
test_CIFAR10.data = test_CIFAR10.data.reshape(10000, 3, 32, 32)

    
reduced_train_CIFAR10 = class_sampling.Reduce(train_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True, transform=transform)
reduced_test_CIFAR10 = class_sampling.Reduce(test_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True, transform=transform)

ratio_train_CIFAR10 = class_sampling.Ratio(train_CIFAR10, NUM_CLASSES_REDUCED, ratio, nums=nums, transform=transform)

triplet_train_CIFAR10 = class_sampling.ForTripletLoss(reduced_train_CIFAR10, smote=False, transform=transform)
triplet_ratio_train_CIFAR10 = class_sampling.ForTripletLoss(ratio_train_CIFAR10, smote=False, transform=transform)

smote_train_CIFAR10 = class_sampling.Smote(ratio_train_CIFAR10, 5000 * NUM_CLASSES_REDUCED, transform=transform)
triplet_smote_train_CIFAR10 = class_sampling.ForTripletLoss(smote_train_CIFAR10, smote=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
targets = ratio_train_CIFAR10.labels 

class_count = np.unique(targets, return_counts=True)[1]
print(class_count)

weight = 1. / class_count

samples_weight = weight[targets]
samples_weight = torch.from_numpy(samples_weight)
oversampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(max(class_count) * NUM_CLASSES_REDUCED), replacement=True)
sampler = torch.utils.data.WeightedRandomSampler(samples_weight, len(samples_weight), replacement=True)
undersampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(min(class_count) * NUM_CLASSES_REDUCED), replacement=False)
undersampler_smote = torch.utils.data.WeightedRandomSampler(samples_weight, int(min(class_count) * 50 * NUM_CLASSES_REDUCED), replacement=False)
weight *= class_count[0]

[5000   50]


In [7]:
train_loader = DataLoader(train_CIFAR10, batch_size=batch_size_train, shuffle=True)  

train_loader_reduced = DataLoader(reduced_train_CIFAR10, batch_size=batch_size_train, shuffle=True)  

train_loader_ratio = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, shuffle=True) 

train_loader_oversampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=oversampler)

train_loader_undersampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=undersampler)

train_loader_sampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=sampler)

train_loader_smote = DataLoader(smote_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_smote_undersampled = DataLoader(smote_train_CIFAR10, batch_size=batch_size_train, sampler=undersampler_smote)

train_loader_tripletloss = DataLoader(triplet_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_tripletloss_ratio = DataLoader(triplet_ratio_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

train_loader_tripletloss_smote = DataLoader(triplet_smote_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

test_loader_reduced = DataLoader(reduced_test_CIFAR10, batch_size=batch_size_test, shuffle=True)

In [24]:
# 2 CLASS normal

momentum=0
learning_rates = [5e-2, 1e-2, 1e-3, 5e-4]

learning_rate_aucs = []


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_reduced, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["normal", 2, nums, (1, 1), learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm]
    rows.append(row)



Test set: Avg. loss: 0.0006989752054214478, AUC: 0.306551



KeyboardInterrupt: 

In [47]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []


In [48]:
# 2 CLASS ratio

momentum=0
learning_rates = [5e-2, 1e-2, 1e-3, 5e-4, 5e-3]

learning_rate_aucs = []
learning_rate_train_aucs = []


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_ratio, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                _, train_auc = metric_utils.auc_sigmoid(train_loader_ratio, network) 
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["ratio", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm]
    rows.append(row)



Test set: Avg. loss: 0.0006964986324310303, AUC: 0.36889099999999997


Test set: Avg. loss: 0.0020484524965286256, AUC: 0.7839530000000001


Test set: Avg. loss: 0.0008004228307837897, AUC: 0.795284


Test set: Avg. loss: 0.0019669443964958192, AUC: 0.8054380000000001


Test set: Avg. loss: 0.0007447379432839922, AUC: 0.845868


Test set: Avg. loss: 0.001964762210845947, AUC: 0.8220825


Test set: Avg. loss: 0.0006831325510387668, AUC: 0.8975879999999999


Test set: Avg. loss: 0.0007002675533294678, AUC: 0.29741149999999994


Test set: Avg. loss: 0.002268509268760681, AUC: 0.7684610000000001


Test set: Avg. loss: 0.000813441066587768, AUC: 0.779452


Test set: Avg. loss: 0.001961479306221008, AUC: 0.8012969999999999


Test set: Avg. loss: 0.0007347470497952239, AUC: 0.8548


Test set: Avg. loss: 0.001975426435470581, AUC: 0.8130399999999999


Test set: Avg. loss: 0.0006815541962548943, AUC: 0.888204


Test set: Avg. loss: 0.0006969135999679566, AUC: 0.396233


Test set: Avg. loss: 0.


Test set: Avg. loss: 0.0022208908796310426, AUC: 0.738314


Test set: Avg. loss: 0.0008432061213188537, AUC: 0.747452


Test set: Avg. loss: 0.0021523762941360472, AUC: 0.7596109999999999


Test set: Avg. loss: 0.0008099005716198152, AUC: 0.772908


Test set: Avg. loss: 0.0006932830810546875, AUC: 0.540304


Test set: Avg. loss: 0.002492153882980347, AUC: 0.69481


Test set: Avg. loss: 0.0008906742870475691, AUC: 0.70906


Test set: Avg. loss: 0.002333818554878235, AUC: 0.721683


Test set: Avg. loss: 0.0008443353652474608, AUC: 0.739868


Test set: Avg. loss: 0.002207614302635193, AUC: 0.743633


Test set: Avg. loss: 0.000816104515907493, AUC: 0.7657440000000001


Test set: Avg. loss: 0.000693915456533432, AUC: 0.470027


Test set: Avg. loss: 0.0019597501158714293, AUC: 0.6219365000000001


Test set: Avg. loss: 0.0010087698723862666, AUC: 0.6026600000000001


Test set: Avg. loss: 0.0025770732164382935, AUC: 0.649457


Test set: Avg. loss: 0.0009415403474583336, AUC: 0.63322


Test se


Test set: Avg. loss: 0.0010332340002059937, AUC: 0.639335


Test set: Avg. loss: 0.002572380872351108, AUC: 0.644908


Test set: Avg. loss: 0.0018369460701942444, AUC: 0.637613


Test set: Avg. loss: 0.0010801145913872387, AUC: 0.641616


Test set: Avg. loss: 0.0022674994468688966, AUC: 0.640206


Test set: Avg. loss: 0.0009589608891470597, AUC: 0.643248


Test set: Avg. loss: 0.0006972042322158814, AUC: 0.3433105


Test set: Avg. loss: 0.0014003113508224488, AUC: 0.540345


Test set: Avg. loss: 0.001675022819844803, AUC: 0.536408


Test set: Avg. loss: 0.002195003390312195, AUC: 0.5878695


Test set: Avg. loss: 0.0009807692990075833, AUC: 0.581788


Test set: Avg. loss: 0.0025299243927001955, AUC: 0.612595


Test set: Avg. loss: 0.0009592226822630013, AUC: 0.60672


Test set: Avg. loss: 0.0006968080401420593, AUC: 0.42184750000000004


Test set: Avg. loss: 0.0014318109154701233, AUC: 0.500686


Test set: Avg. loss: 0.0016474896108750069, AUC: 0.46987599999999996


Test set: Avg. loss

In [49]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [14]:
# 2 CLASS oversampled

momentum=0
learning_rates = [5e-2, 1e-2, 1e-3, 5e-3, 1e-4]

learning_rate_aucs = []


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_oversampled, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["oversampled", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm]
    rows.append(row)


Test set: Avg. loss: 0.0006984597146511078, AUC: 0.3728135



KeyboardInterrupt: 

In [51]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [52]:
# 2 CLASS undersampled

momentum=0
learning_rates = [5e-2, 1e-2, 1e-3, 5e-3, 1e-4]

learning_rate_aucs = []


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_undersampled, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["undersampled", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm]
    rows.append(row)



Test set: Avg. loss: 0.0006928627490997315, AUC: 0.5610565


Test set: Avg. loss: 0.0006705484390258789, AUC: 0.731087


Test set: Avg. loss: 0.0006308182775974274, AUC: 0.7689


Test set: Avg. loss: 0.0006022096574306488, AUC: 0.792463


Test set: Avg. loss: 0.0006950775682926178, AUC: 0.4702115


Test set: Avg. loss: 0.0006825314462184907, AUC: 0.7292394999999999


Test set: Avg. loss: 0.0006524708569049835, AUC: 0.754748


Test set: Avg. loss: 0.0006467648446559906, AUC: 0.7728849999999999


Test set: Avg. loss: 0.0006955726444721221, AUC: 0.3766905


Test set: Avg. loss: 0.0007012077569961548, AUC: 0.601467


Test set: Avg. loss: 0.0006722729504108429, AUC: 0.7478565


Test set: Avg. loss: 0.0006584925651550293, AUC: 0.764774


Test set: Avg. loss: 0.0006918634176254272, AUC: 0.548633


Test set: Avg. loss: 0.0006640854477882385, AUC: 0.7368910000000001


Test set: Avg. loss: 0.0006435896456241608, AUC: 0.772193


Test set: Avg. loss: 0.0006132080554962159, AUC: 0.805193


Test se


Test set: Avg. loss: 0.000681488573551178, AUC: 0.661619


Test set: Avg. loss: 0.0006757974326610565, AUC: 0.6790075000000001


Test set: Avg. loss: 0.0006896138191223144, AUC: 0.610887


Test set: Avg. loss: 0.0006893069446086884, AUC: 0.613855


Test set: Avg. loss: 0.0006885486543178558, AUC: 0.633282


Test set: Avg. loss: 0.0006888088881969452, AUC: 0.6441379999999999


Test set: Avg. loss: 0.000698607712984085, AUC: 0.47644650000000005


Test set: Avg. loss: 0.0006970592141151428, AUC: 0.5706005000000001


Test set: Avg. loss: 0.0006956733465194702, AUC: 0.6301249999999999


Test set: Avg. loss: 0.0006951669156551361, AUC: 0.6572


Test set: Avg. loss: 0.0006925795376300812, AUC: 0.575475


Test set: Avg. loss: 0.0006951481699943542, AUC: 0.5105139999999999


Test set: Avg. loss: 0.0006970000267028808, AUC: 0.4651495


Test set: Avg. loss: 0.000699210911989212, AUC: 0.424679


Test set: Avg. loss: 0.0006938398480415344, AUC: 0.49060099999999995


Test set: Avg. loss: 0.00069005

In [53]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [54]:
# 2 Class Weighted Loss 

momentum=0
learning_rates = [5e-2, 1e-2, 5e-3, 1e-3, 5e-4]

learning_rate_aucs = []

loss_fn_args = {}
loss_fn_args['pos_weight'] = torch.tensor([weight[1]])


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_ratio, network, optimizer, verbose=False, loss_fn_args=loss_fn_args)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                _, train_auc = metric_utils.auc_sigmoid(train_loader_ratio, network) 
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["weighted", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm]
    rows.append(row)



Test set: Avg. loss: 0.0006929865777492523, AUC: 0.6035210000000001


Test set: Avg. loss: 0.0006941011548042297, AUC: 0.5


Test set: Avg. loss: 0.01152815311261923, AUC: 0.5


Test set: Avg. loss: 0.0006933517456054688, AUC: 0.5


Test set: Avg. loss: 0.010536302033037242, AUC: 0.5


Test set: Avg. loss: 0.0006934399306774139, AUC: 0.5


Test set: Avg. loss: 0.01047680665950964, AUC: 0.5


Test set: Avg. loss: 0.0006829564273357391, AUC: 0.7606660000000001


Test set: Avg. loss: 0.0007110981047153472, AUC: 0.513034


Test set: Avg. loss: 0.008160775073684089, AUC: 0.4939


Test set: Avg. loss: 0.0006991162598133087, AUC: 0.5


Test set: Avg. loss: 0.009259643141586001, AUC: 0.5


Test set: Avg. loss: 0.0006932158470153809, AUC: 0.5


Test set: Avg. loss: 0.011024211881184342, AUC: 0.5


Test set: Avg. loss: 0.0006900024712085724, AUC: 0.6275345


Test set: Avg. loss: 0.0006032556593418122, AUC: 0.6915875


Test set: Avg. loss: 0.009716101306499822, AUC: 0.67254


Test set: Avg. loss


Test set: Avg. loss: 0.0005637988150119782, AUC: 0.828475


Test set: Avg. loss: 0.00714115304521995, AUC: 0.9039240000000001


Test set: Avg. loss: 0.0005115714669227601, AUC: 0.872054


Test set: Avg. loss: 0.0058704357690150194, AUC: 0.9506


Test set: Avg. loss: 0.0005990622639656067, AUC: 0.854916


Test set: Avg. loss: 0.004999826568188054, AUC: 0.9478759999999999


Test set: Avg. loss: 0.0006893006265163422, AUC: 0.592972


Test set: Avg. loss: 0.0005413131713867187, AUC: 0.838742


Test set: Avg. loss: 0.009401522638774154, AUC: 0.873108


Test set: Avg. loss: 0.0005082505941390992, AUC: 0.8643450000000001


Test set: Avg. loss: 0.007597315765843533, AUC: 0.93034


Test set: Avg. loss: 0.0005349463224411011, AUC: 0.819852


Test set: Avg. loss: 0.005960197472336269, AUC: 0.952184


Test set: Avg. loss: 0.0006917398273944855, AUC: 0.5357335000000001


Test set: Avg. loss: 0.0005320677161216736, AUC: 0.872421


Test set: Avg. loss: 0.007882021895729669, AUC: 0.885336


Test set:


Test set: Avg. loss: 0.0005055390298366547, AUC: 0.849865


Test set: Avg. loss: 0.008350928868397627, AUC: 0.8736079999999999


Test set: Avg. loss: 0.0006963136792182922, AUC: 0.39968349999999997


Test set: Avg. loss: 0.0005984225273132324, AUC: 0.771389


Test set: Avg. loss: 0.009001542176350508, AUC: 0.78562


Test set: Avg. loss: 0.000558992862701416, AUC: 0.8067799999999999


Test set: Avg. loss: 0.008595412568290635, AUC: 0.829488


Test set: Avg. loss: 0.000538530021905899, AUC: 0.82298


Test set: Avg. loss: 0.007287021987509019, AUC: 0.8586119999999999


Test set: Avg. loss: 0.0006968409717082977, AUC: 0.38355749999999994


Test set: Avg. loss: 0.0006436823904514313, AUC: 0.803137


Test set: Avg. loss: 0.01135210377154964, AUC: 0.7909520000000001


Test set: Avg. loss: 0.000583594560623169, AUC: 0.8077380000000001


Test set: Avg. loss: 0.01014432730060993, AUC: 0.820932


Test set: Avg. loss: 0.0005280269980430603, AUC: 0.833491


Test set: Avg. loss: 0.00862787301587586

In [55]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [56]:
# 2 CLASS SMOTE

momentum=0
learning_rates = [5e-2, 1e-2, 5e-3, 1e-3, 1e-4]

learning_rate_aucs = []

loss_fn_args = {}
loss_fn_args['loss_cap'] = None


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid_with_smote(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn_args=loss_fn_args)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["smote", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm]
    rows.append(row)



Test set: Avg. loss: 0.0006901857256889344, AUC: 0.6220315000000001


Test set: Avg. loss: 0.0008553669452667236, AUC: 0.9138239999999999


Test set: Avg. loss: 0.0014686028361320495, AUC: 0.9061444999999999


Test set: Avg. loss: 0.0017553408741950989, AUC: 0.886091


Test set: Avg. loss: 0.0006922211647033691, AUC: 0.565235


Test set: Avg. loss: 0.001127453625202179, AUC: 0.8912439999999999


Test set: Avg. loss: 0.0015964934825897217, AUC: 0.880198


Test set: Avg. loss: 0.0013375398516654968, AUC: 0.885392


Test set: Avg. loss: 0.000696012943983078, AUC: 0.603688


Test set: Avg. loss: 0.0007503098547458649, AUC: 0.892879


Test set: Avg. loss: 0.00129351145029068, AUC: 0.9026019999999999


Test set: Avg. loss: 0.0012972863912582397, AUC: 0.855303


Test set: Avg. loss: 0.0006966092586517334, AUC: 0.5215055


Test set: Avg. loss: 0.0009291221201419831, AUC: 0.8892810000000001


Test set: Avg. loss: 0.0008040020763874054, AUC: 0.898658


Test set: Avg. loss: 0.0017433229088783264


Test set: Avg. loss: 0.000618213951587677, AUC: 0.7882954999999999


Test set: Avg. loss: 0.0005512105226516724, AUC: 0.8245439999999999


Test set: Avg. loss: 0.0005293850600719452, AUC: 0.847711


Test set: Avg. loss: 0.0006969168782234192, AUC: 0.413656


Test set: Avg. loss: 0.000612224131822586, AUC: 0.765173


Test set: Avg. loss: 0.0005718506872653961, AUC: 0.7946544999999999


Test set: Avg. loss: 0.0005359521210193634, AUC: 0.8355079999999999


Test set: Avg. loss: 0.0006935161054134369, AUC: 0.5336654999999999


Test set: Avg. loss: 0.000620194137096405, AUC: 0.757107


Test set: Avg. loss: 0.0005708503723144532, AUC: 0.788429


Test set: Avg. loss: 0.000546052247285843, AUC: 0.8274469999999999


Test set: Avg. loss: 0.0006930411756038666, AUC: 0.5264515000000001


Test set: Avg. loss: 0.0006314632296562195, AUC: 0.7641144999999999


Test set: Avg. loss: 0.0005780141651630402, AUC: 0.791547


Test set: Avg. loss: 0.0005373639762401581, AUC: 0.834433


Test set: Avg. loss: 0.

In [57]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [9]:
# 2 Class Capped SMOTE 

momentum=0
learning_rates = [1e-3, 5e-4, 1e-4]


cap_aucs = []

caps = [1, 5, 10]

for cap in caps:
    
    loss_fn_args = {}
    loss_fn_args['loss_cap'] = cap
    
    learning_rate_aucs = []

    for learning_rate in learning_rates:
        aucs = []
        for i in range(10):
            model_aucs = []
            network = models.ConvNet(NUM_CLASSES_REDUCED)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
            model_aucs.append(auc)
            for epoch in range(n_epochs):
                _, _ = train.train_sigmoid_with_smote(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)

    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    
    
    cap_aucs.append([auc_mean, auc_variance])

    
    
for i in range(len(cap_aucs)):
    auc_mean = cap_aucs[i][0]
    auc_variance = cap_aucs[i][1]
    cap = caps[i]
    for i in range(len(learning_rates)): 
        row = ["capped_smote", 2, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm]
        rows.append(row)

print(rows)



Test set: Avg. loss: 0.0008589592576026916, AUC: 0.610939


Test set: Avg. loss: 0.0014835682511329651, AUC: 0.7226130000000002


Test set: Avg. loss: 0.001802017629146576, AUC: 0.747778


Test set: Avg. loss: 0.0016669080257415772, AUC: 0.7606769999999999


Test set: Avg. loss: 0.0014676600098609924, AUC: 0.632443


Test set: Avg. loss: 0.0011303189992904663, AUC: 0.767101


Test set: Avg. loss: 0.0008886577188968658, AUC: 0.8482649999999998


Test set: Avg. loss: 0.0010313032269477845, AUC: 0.855079


Test set: Avg. loss: 0.0011315805912017823, AUC: 0.626282


Test set: Avg. loss: 0.001719098687171936, AUC: 0.8240619999999999


Test set: Avg. loss: 0.0010683817267417909, AUC: 0.861991


Test set: Avg. loss: 0.0009965504705905914, AUC: 0.873041


Test set: Avg. loss: 0.001854050636291504, AUC: 0.549836


Test set: Avg. loss: 0.0016986330151557922, AUC: 0.7641640000000001


Test set: Avg. loss: 0.0016748768091201782, AUC: 0.8111499999999999


Test set: Avg. loss: 0.0008317158222198486

KeyboardInterrupt: 

In [ ]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = (col_names))

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [ ]:
# 2 CLASS Focal Loss

momentum=0
learning_rates = [1e-4, 1e-5]

learning_rate_aucs = []

loss_fn_args = {}


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_ratio, network, optimizer, verbose=False, loss_fn=loss_fns.SigmoidFocalLoss, loss_fn_args=loss_fn_args)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["focal_loss", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)


In [38]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

NameError: name 'df2' is not defined

In [12]:
# distance + capped loss
momentum=0
learning_rates = [1e-3, 5e-3, 5e-4, 1e-4, 5e-5]


    
loss_fn_args = {}
loss_cap = 1.0
loss_fn_args['distance'] = 'euclidean'

start_epoch = 2

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNetWithEmbeddings(2)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True) 
        model_aucs.append(auc)
        for epoch in range(start_epoch):
            loss_fn_args['loss_cap'] = None
            _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
        for epoch in range(start_epoch, n_epochs + 1):
            loss_fn_args['loss_cap'] = loss_cap
            _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)


for i in range(len(learning_rates)): 
    row = ["distance_capped_smote", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], loss_fn_args['loss_cap'], norm]
    rows.append(row)


Test set: Avg. loss: 0.0007034576535224914, AUC: 0.350273


Test set: Avg. loss: 0.0005969953835010529, AUC: 0.770484


Test set: Avg. loss: 0.000597990334033966, AUC: 0.7816185


Test set: Avg. loss: 0.0006023451387882232, AUC: 0.7982774999999999


Test set: Avg. loss: 0.0006960260272026062, AUC: 0.36476299999999995


Test set: Avg. loss: 0.0006060459017753602, AUC: 0.768943


Test set: Avg. loss: 0.0005872988700866699, AUC: 0.7846535000000001


Test set: Avg. loss: 0.0005800300836563111, AUC: 0.8063929999999999


Test set: Avg. loss: 0.0006880538761615754, AUC: 0.6572255


Test set: Avg. loss: 0.0006098485887050629, AUC: 0.7543479999999999


Test set: Avg. loss: 0.0005866237282752991, AUC: 0.790246


Test set: Avg. loss: 0.0005785414576530457, AUC: 0.820869


Test set: Avg. loss: 0.0006959007680416107, AUC: 0.437869


Test set: Avg. loss: 0.0006432939171791077, AUC: 0.7988230000000001


Test set: Avg. loss: 0.0005786927938461303, AUC: 0.792296


Test set: Avg. loss: 0.00058129939436


Test set: Avg. loss: 0.0006857488453388214, AUC: 0.6640845


Test set: Avg. loss: 0.0006802918016910553, AUC: 0.6844085


Test set: Avg. loss: 0.0006736926138401031, AUC: 0.696304


Test set: Avg. loss: 0.0006890509724617004, AUC: 0.6708350000000001


Test set: Avg. loss: 0.0006851511299610138, AUC: 0.744216


Test set: Avg. loss: 0.0006813865900039673, AUC: 0.7735485


Test set: Avg. loss: 0.0006771237552165985, AUC: 0.7873035


Test set: Avg. loss: 0.0006936484277248382, AUC: 0.47479800000000005


Test set: Avg. loss: 0.0006836704015731811, AUC: 0.7268295


Test set: Avg. loss: 0.000674445629119873, AUC: 0.7734519999999999


Test set: Avg. loss: 0.0006638656854629516, AUC: 0.7779069999999999


Test set: Avg. loss: 0.0006897411644458771, AUC: 0.6272595


Test set: Avg. loss: 0.0006844063699245452, AUC: 0.7086825000000001


Test set: Avg. loss: 0.0006791321933269501, AUC: 0.72961


Test set: Avg. loss: 0.0006723710894584655, AUC: 0.7380399999999999


Test set: Avg. loss: 0.00069241988

KeyboardInterrupt: 

In [19]:
for i in range(len(learning_rates) - 1): 
    row = ["distance_capped_smote", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], loss_fn_args['loss_cap'], norm]
    rows.append(row)

In [21]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [ ]:
# cosine distance + capped loss
momentum=0
learning_rates = [1e-3, 5e-3, 5e-4, 1e-4, 5e-5]


    
loss_fn_args = {}
loss_fn_args['loss_cap'] = 5.0
loss_fn_args['distance'] = 'cosine'

start_epoch = 2

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNetWithEmbeddings(2)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True) 
        model_aucs.append(auc)
        for epoch in range(start_epoch):
            loss_fn_args['loss_cap'] = None
            _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
        for epoch in range(start_epoch, n_epochs + 1):
            _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)


for i in range(len(learning_rates)): 
    row = ["cosine_distance_capped_smote", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], loss_fn_args['loss_cap'], norm]
    rows.append(row)

In [ ]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [16]:
# 2 class triplet loss no ratio 
# no smote 

# note: sometimes can get a very high accuracy but may diverge

momentum=0
learning_rates = [(5e-6, 1e-3), (1e-6, 5e-4), (1e-6, 1e-4), (5e-6, 5e-4)]

learning_rate_aucs = []

    

for learning_rate in learning_rates:
    aucs = []
    for i in range(10): 
        model_aucs = []
        embed_network = models.ConvNetOnlyEmbeddings(2)
        linear_probe = models.ConvNetLinearProbe(2)
        complete_network = models.CompleteConvNet(embed_network, linear_probe)
        embed_optimizer = optim.SGD(embed_network.parameters(), lr=learning_rate[0], momentum=momentum)
        linear_optimizer = optim.SGD(complete_network.parameters(), lr=learning_rate[1], momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True) 
        model_aucs.append(auc)
        for epoch in range(15):
            _, train_losses = train.train_triplet_loss(epoch, train_loader_tripletloss, embed_network, embed_optimizer, verbose=False)
            print("Train loss: " + str(np.mean(np.array(train_losses))))
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_reduced, complete_network, linear_optimizer, verbose=False)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True)
            if (epoch + 1) % 10 == 0: 
                model_aucs.append(auc) 
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)


learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["triplet_loss", 2, nums, (1, 1), learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], 0.0, norm]
    rows.append(row)


Test set: Avg. loss: 0.0006970875263214111, AUC: 0.5179245

Train loss: 0.27050119904196185
Train loss: 0.2508804725993211
Train loss: 0.2295536493799489
Train loss: 0.23650908014576905
Train loss: 0.26522073852028816
Train loss: 0.2877479203187736
Train loss: 0.24342828267698835
Train loss: 0.2656087279319763
Train loss: 0.24802333306355082
Train loss: 0.23219373499511914
Train loss: 0.23283238889305455
Train loss: 0.2749264088405925
Train loss: 0.23505860794881347
Train loss: 0.22109104569550533
Train loss: 0.22770290124188564

Test set: Avg. loss: 0.0006762307286262512, AUC: 0.744272


Test set: Avg. loss: 0.0006601359844207764, AUC: 0.774519


Test set: Avg. loss: 0.0006447486281394958, AUC: 0.78154


Test set: Avg. loss: 0.0006311907172203064, AUC: 0.786457


Test set: Avg. loss: 0.0006190038323402405, AUC: 0.7906310000000001


Test set: Avg. loss: 0.0006070325672626496, AUC: 0.795851


Test set: Avg. loss: 0.0005986305773258209, AUC: 0.8024519999999999


Test set: Avg. loss: 0.0


Test set: Avg. loss: 0.0006099905967712402, AUC: 0.7951569999999999


Test set: Avg. loss: 0.000596579909324646, AUC: 0.7971199999999999


Test set: Avg. loss: 0.0005863858163356781, AUC: 0.801263


Test set: Avg. loss: 0.00057879638671875, AUC: 0.80675


Test set: Avg. loss: 0.000572771966457367, AUC: 0.811958


Test set: Avg. loss: 0.0005666111409664154, AUC: 0.8163039999999999


Test set: Avg. loss: 0.000559524118900299, AUC: 0.8201769999999999


Test set: Avg. loss: 0.0005542269051074982, AUC: 0.8243260000000001


Test set: Avg. loss: 0.0005485452115535736, AUC: 0.8279209999999999


Test set: Avg. loss: 0.0005436902046203613, AUC: 0.832715


Test set: Avg. loss: 0.0005399761497974396, AUC: 0.8367309999999999


Test set: Avg. loss: 0.0005338923335075379, AUC: 0.838973


Test set: Avg. loss: 0.0005322109758853912, AUC: 0.8437689999999999


Test set: Avg. loss: 0.0005274032056331635, AUC: 0.8485179999999999


Test set: Avg. loss: 0.0005236106216907501, AUC: 0.8523729999999999


Test 


Test set: Avg. loss: 0.0005457632243633271, AUC: 0.8351440000000001


Test set: Avg. loss: 0.0005399615466594696, AUC: 0.8389530000000001


Test set: Avg. loss: 0.0005355960130691528, AUC: 0.842231


Test set: Avg. loss: 0.0005317294597625733, AUC: 0.848512


Test set: Avg. loss: 0.0005271710455417633, AUC: 0.8521449999999999


Test set: Avg. loss: 0.0005212545394897461, AUC: 0.8550009999999999


Test set: Avg. loss: 0.0005165989398956299, AUC: 0.859765


Test set: Avg. loss: 0.0005129876136779786, AUC: 0.863526


Test set: Avg. loss: 0.0005102378427982331, AUC: 0.86945


Test set: Avg. loss: 0.0005042073428630829, AUC: 0.871496


Test set: Avg. loss: 0.0004981806427240372, AUC: 0.874105


Test set: Avg. loss: 0.0004947265684604645, AUC: 0.8766989999999999


Test set: Avg. loss: 0.0004910622239112854, AUC: 0.879653


Test set: Avg. loss: 0.0004856421500444412, AUC: 0.882077


Test set: Avg. loss: 0.0004833685904741287, AUC: 0.8857980000000001


Test set: Avg. loss: 0.00068809446692466


Test set: Avg. loss: 0.00048418672382831576, AUC: 0.8903570000000001


Test set: Avg. loss: 0.0007154200375080108, AUC: 0.49844649999999996

Train loss: 0.29820947472456916
Train loss: 0.19961200132491483
Train loss: 0.2934751742205043
Train loss: 0.3064685365197006
Train loss: 0.24654365648889237
Train loss: 0.2999751810814924
Train loss: 0.28235246193636754
Train loss: 0.30309595537793105
Train loss: 0.3140808393241494
Train loss: 0.30847976124210724
Train loss: 0.23305495833135714
Train loss: 0.25309754784699456
Train loss: 0.25631907297547457
Train loss: 0.21590502740471226
Train loss: 0.3219998133410314

Test set: Avg. loss: 0.0006911126673221588, AUC: 0.650932


Test set: Avg. loss: 0.0006829962134361267, AUC: 0.7456495


Test set: Avg. loss: 0.0006768400073051453, AUC: 0.7868360000000001


Test set: Avg. loss: 0.0006702893078327179, AUC: 0.801085


Test set: Avg. loss: 0.0006626879274845123, AUC: 0.8048610000000003


Test set: Avg. loss: 0.0006542136073112488, AUC: 0.804765


T


Test set: Avg. loss: 0.0006245045065879822, AUC: 0.784751


Test set: Avg. loss: 0.0006122897863388062, AUC: 0.790363


Test set: Avg. loss: 0.0006018477082252502, AUC: 0.793475


Test set: Avg. loss: 0.0005927428007125855, AUC: 0.796139


Test set: Avg. loss: 0.0005850924253463745, AUC: 0.798675


Test set: Avg. loss: 0.0005791719257831573, AUC: 0.8010760000000001


Test set: Avg. loss: 0.0005736169219017029, AUC: 0.8044505


Test set: Avg. loss: 0.0005692993998527527, AUC: 0.8074489999999999


Test set: Avg. loss: 0.0005645809173583984, AUC: 0.8098320000000001


Test set: Avg. loss: 0.0005614774823188782, AUC: 0.8125439999999999


Test set: Avg. loss: 0.0005582036674022674, AUC: 0.8153499999999998


Test set: Avg. loss: 0.0005544720292091369, AUC: 0.818101


Test set: Avg. loss: 0.0005512963831424714, AUC: 0.821227


Test set: Avg. loss: 0.0005466761589050293, AUC: 0.824775


Test set: Avg. loss: 0.000542959064245224, AUC: 0.827258


Test set: Avg. loss: 0.0005388966798782348, AUC: 

...


Test set: Avg. loss: 0.0006680652797222137, AUC: 0.76432


Test set: Avg. loss: 0.0006656407713890076, AUC: 0.7686085


Test set: Avg. loss: 0.0006630861163139343, AUC: 0.771946


Test set: Avg. loss: 0.0006605594158172607, AUC: 0.7745869999999999


Test set: Avg. loss: 0.0006580725610256195, AUC: 0.776539


Test set: Avg. loss: 0.0006558189690113068, AUC: 0.7780509999999999


Test set: Avg. loss: 0.0006531240344047546, AUC: 0.7795895000000002


Test set: Avg. loss: 0.0006505233943462372, AUC: 0.7808269999999999


Test set: Avg. loss: 0.0006481728553771973, AUC: 0.7818244999999999


Test set: Avg. loss: 0.0006457369327545166, AUC: 0.782664


Test set: Avg. loss: 0.0006431747376918793, AUC: 0.7834165


Test set: Avg. loss: 0.0006409286558628082, AUC: 0.7842640000000001


Test set: Avg. loss: 0.0006386128962039948, AUC: 0.7848859999999999


Test set: Avg. loss: 0.0006362033784389496, AUC: 0.785704


Test set: Avg. loss: 0.0006338813006877899, AUC: 0.786359


Test set: Avg. loss: 0.00063


Test set: Avg. loss: 0.0006704702079296112, AUC: 0.6895560000000001


Test set: Avg. loss: 0.0006694221794605255, AUC: 0.6925425


Test set: Avg. loss: 0.0006682207584381104, AUC: 0.695418


Test set: Avg. loss: 0.0006670372188091278, AUC: 0.6981189999999999


Test set: Avg. loss: 0.000665871649980545, AUC: 0.700533


Test set: Avg. loss: 0.0006646739840507508, AUC: 0.703076


Test set: Avg. loss: 0.0006634033322334289, AUC: 0.7056069999999999


Test set: Avg. loss: 0.0006621023416519165, AUC: 0.7081339999999999


Test set: Avg. loss: 0.0006607334017753601, AUC: 0.7107690000000001


Test set: Avg. loss: 0.0006594657003879547, AUC: 0.713323


Test set: Avg. loss: 0.0006581801772117615, AUC: 0.7155935


Test set: Avg. loss: 0.0006569226384162903, AUC: 0.7179595000000001


Test set: Avg. loss: 0.0006557233035564422, AUC: 0.720136


Test set: Avg. loss: 0.0006544747352600097, AUC: 0.7222489999999999


Test set: Avg. loss: 0.0006923116147518158, AUC: 0.5312720000000001

Train loss: 0.25660


Test set: Avg. loss: 0.0006405390202999115, AUC: 0.792479


Test set: Avg. loss: 0.000718257486820221, AUC: 0.282124

Train loss: 0.24279541650395484
Train loss: 0.21561232797659127
Train loss: 0.21026910907903296
Train loss: 0.22105312119623657
Train loss: 0.24160270326456446
Train loss: 0.20978607265812577
Train loss: 0.20176735994922126
Train loss: 0.1943474138618275
Train loss: 0.19081071114084522
Train loss: 0.1840556727093496
Train loss: 0.19481274162887768
Train loss: 0.1982407649611212
Train loss: 0.21892993047738532
Train loss: 0.18922295160354322
Train loss: 0.16525371553032261

Test set: Avg. loss: 0.0006855151653289795, AUC: 0.6416220000000001


Test set: Avg. loss: 0.0006722566485404968, AUC: 0.7267460000000001


Test set: Avg. loss: 0.0006618347465991974, AUC: 0.745714


Test set: Avg. loss: 0.0006519446969032287, AUC: 0.752426


Test set: Avg. loss: 0.0006433593630790711, AUC: 0.7565330000000001


Test set: Avg. loss: 0.0006352792382240296, AUC: 0.760345


Test set: Avg


Test set: Avg. loss: 0.0006858068108558655, AUC: 0.6743560000000001


Test set: Avg. loss: 0.0006824159622192383, AUC: 0.710034


Test set: Avg. loss: 0.000678071141242981, AUC: 0.7358359999999999


Test set: Avg. loss: 0.0006730778515338897, AUC: 0.7509064999999999


Test set: Avg. loss: 0.0006670894026756287, AUC: 0.7610525000000001


Test set: Avg. loss: 0.000660363346338272, AUC: 0.7685660000000001


Test set: Avg. loss: 0.0006531411111354827, AUC: 0.7724639999999999


Test set: Avg. loss: 0.0006443365514278412, AUC: 0.7746190000000001


Test set: Avg. loss: 0.0006359018683433533, AUC: 0.7765575


Test set: Avg. loss: 0.0006282975375652314, AUC: 0.778751


Test set: Avg. loss: 0.0006207510530948639, AUC: 0.7814180000000001


Test set: Avg. loss: 0.0006139645278453827, AUC: 0.782877


Test set: Avg. loss: 0.0006078597009181976, AUC: 0.7846439999999999


Test set: Avg. loss: 0.0006020857095718384, AUC: 0.786891


Test set: Avg. loss: 0.0005966466069221496, AUC: 0.789455


Test set: 


Test set: Avg. loss: 0.0005665676593780518, AUC: 0.8105120000000001


Test set: Avg. loss: 0.000561883419752121, AUC: 0.814373


Test set: Avg. loss: 0.00055844646692276, AUC: 0.818042


Test set: Avg. loss: 0.0005541107356548309, AUC: 0.821893


Test set: Avg. loss: 0.0005503147840499878, AUC: 0.8247199999999999


Test set: Avg. loss: 0.0005475920438766479, AUC: 0.827519


Test set: Avg. loss: 0.0005445378422737121, AUC: 0.830985


Test set: Avg. loss: 0.0005404637753963471, AUC: 0.8352550000000001


Test set: Avg. loss: 0.0005381945669651031, AUC: 0.8373069999999999


Test set: Avg. loss: 0.0005364443063735962, AUC: 0.840086


Test set: Avg. loss: 0.000533218115568161, AUC: 0.843809


Test set: Avg. loss: 0.0005298088490962983, AUC: 0.8462309999999998


Test set: Avg. loss: 0.0005272234380245209, AUC: 0.848664


Test set: Avg. loss: 0.0005233173668384552, AUC: 0.851064


Test set: Avg. loss: 0.0006973285675048828, AUC: 0.5184565

Train loss: 0.27460544997719444
Train loss: 0.2617756

In [17]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [11]:
# 2 class triplet loss with capped SMOTE 

# note: sometimes can get a very high accuracy but may diverge

momentum=0
learning_rates = [(5e-6, 1e-3), (1e-6, 5e-4), (1e-6, 1e-4), (5e-6, 5e-4)]

learning_rate_aucs = []

loss_fn_args = {}
loss_fn_args['loss_cap'] = 5.0
loss_fn_args['distance'] = 'euclidean'


for learning_rate in learning_rates:
    aucs = []
    for i in range(10): 
        model_aucs = []
        embed_network = models.ConvNetOnlyEmbeddings(2)
        linear_probe = models.ConvNetLinearProbe(2)
        complete_network = models.CompleteConvNet(embed_network, linear_probe)
        embed_optimizer = optim.SGD(embed_network.parameters(), lr=learning_rate[0], momentum=momentum)
        linear_optimizer = optim.SGD(complete_network.parameters(), lr=learning_rate[1], momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True) 
        model_aucs.append(auc)
        for epoch in range(15):
            _, train_losses = train.train_triplet_loss_smote(epoch, train_loader_tripletloss_smote, embed_network, embed_optimizer, verbose=False)
            print("Train loss: " + str(np.mean(np.array(train_losses))))
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_reduced, complete_network, linear_optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True)
            if (epoch + 1) % 10 == 0: 
                model_aucs.append(auc) 
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)


learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["triplet_loss", 2, nums, (1, 1), learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], 0.0, norm]
    rows.append(row)


Test set: Avg. loss: 0.000679101675748825, AUC: 0.7387214999999999



RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor []], which is output 0 of ReluBackward0, is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

In [11]:
# 2 class triplet loss with ratio 
# no smote 

momentum=0
learning_rates = [(1e-7, 1e-5)]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(3): 
        model_aucs = []
        embed_network = models.ConvNetOnlyEmbeddings(2)
        linear_probe = models.ConvNetLinearProbe(2)
        complete_network = models.CompleteConvNet(embed_network, linear_probe)
        embed_optimizer = optim.SGD(embed_network.parameters(), lr=learning_rate[0], momentum=momentum)
        linear_optimizer = optim.SGD(complete_network.parameters(), lr=learning_rate[1], momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True) 
        model_aucs.append(auc)
        for epoch in range(15):
            _, train_losses = train.train_triplet_loss(epoch, train_loader_tripletloss_ratio, embed_network, embed_optimizer, verbose=False)
            print("Train loss: " + str(np.mean(np.array(train_losses))))
        for epoch in range(50):
            _, _ = train.train_linear_probe(epoch, train_loader_reduced, complete_network, linear_optimizer, verbose=False)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, complete_network, embeddings=True)
            if (epoch + 1) % 10 == 0: 
                model_aucs.append(auc) 
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)


learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["triplet_loss", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], 
           auc_mean[i][4], auc_variance[i][4],
           auc_mean[i][5], auc_variance[i][5],
           None]
    rows.append(row)


Test set: Avg. loss: 0.0007769797742366791, AUC: 0.581019

Train loss: 16.096790712090986
Train loss: 7.95101783849016
Train loss: 1.6742434924161886
Train loss: 0.0
Train loss: 6.369154097158698
Train loss: 1.0685905746266813
Train loss: 0.9919730077815961
Train loss: 4.3510085480122624
Train loss: 0.0
Train loss: 0.0
Train loss: 0.0
Train loss: 1.729099128819719
Train loss: 0.0
Train loss: 2.9074361294130737
Train loss: 0.0

Test set: Avg. loss: 0.0007081505060195923, AUC: 0.595766


Test set: Avg. loss: 0.0006918745040893554, AUC: 0.599685


Test set: Avg. loss: 0.0006822091042995453, AUC: 0.606827


Test set: Avg. loss: 0.0006758022308349609, AUC: 0.614831


Test set: Avg. loss: 0.0006717154681682586, AUC: 0.620797


Test set: Avg. loss: 0.0006670982539653778, AUC: 0.6299465


Test set: Avg. loss: 0.0006633270978927612, AUC: 0.641555


Test set: Avg. loss: 0.0006615970730781555, AUC: 0.647514


Test set: Avg. loss: 0.0006594435572624207, AUC: 0.6563729999999999


Test set: Avg. lo


Test set: Avg. loss: 0.0006901566684246063, AUC: 0.5230110000000001


Test set: Avg. loss: 0.0006899235546588897, AUC: 0.5261325


Test set: Avg. loss: 0.0006896623373031616, AUC: 0.5253125


Test set: Avg. loss: 0.0006894396543502808, AUC: 0.5309490000000001


Test set: Avg. loss: 0.0006897644698619842, AUC: 0.52842


Test set: Avg. loss: 0.0006899001598358154, AUC: 0.525083


Test set: Avg. loss: 0.0006898137331008911, AUC: 0.5242764999999999


Test set: Avg. loss: 0.0006899007260799408, AUC: 0.5234755


Test set: Avg. loss: 0.0006897071897983551, AUC: 0.5284414999999999


Test set: Avg. loss: 0.0006894522905349732, AUC: 0.5316019999999999


Test set: Avg. loss: 0.0006894703507423401, AUC: 0.5321385000000001


Test set: Avg. loss: 0.0006892255246639252, AUC: 0.5359525


Test set: Avg. loss: 0.0006892721951007843, AUC: 0.5328990000000001


Test set: Avg. loss: 0.0006891983449459076, AUC: 0.537638


Test set: Avg. loss: 0.0006890941560268402, AUC: 0.5369265000000001


Test set: Avg. l

In [12]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

ValueError: 14 columns passed, passed data had 18 columns

In [31]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [9]:
# capped smote using triplet loss

momentum=0
learning_rates = [5e-3, 1e-3, 1e-2]

loss_fn_args = {}


learning_rate_aucs = []

start_epoch = 2

loss_caps = [1, 5, 10]

for loss_cap in loss_caps:
    for learning_rate in learning_rates:
        aucs = []
        for i in range(5): 
            print(loss_cap)
            model_aucs = []
            network = models.ConvNetWithEmbeddings(2)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(start_epoch):
                loss_fn_args['loss_cap'] = None
                _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
            for epoch in range(start_epoch, n_epochs + 1):
                loss_fn_args['loss_cap'] = loss_cap
                _, _ = train.train_triplet_capped_loss(epoch, train_loader_tripletloss_smote, network, optimizer, verbose=False, cap_calc=loss_fns.TripletLoss,loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)


learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for loss_cap in loss_caps:
    for i in range(len(learning_rates)): 
        row = ["triplet_loss_capped_smote", 2, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], loss_cap, norm]
        rows.append(row)

1

Test set: Avg. loss: 0.0006894645094871521, AUC: 0.6804574999999999



KeyboardInterrupt: 

In [ ]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [1]:
# capped smote using triplet loss w/ average 

momentum=0
learning_rates = [5e-3, 1e-3, 1e-2]

loss_fn_args = {}


learning_rate_aucs = []

start_epoch = 2

loss_caps = [1, 5, 10]
cap_calc = loss_fns.TripletLossWithAverage

for loss_cap in loss_caps:
    for learning_rate in learning_rates:
        aucs = []
        for i in range(5): 
            print(loss_cap)
            model_aucs = []
            network = models.ConvNetWithEmbeddings(2)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(start_epoch):
                loss_fn_args['loss_cap'] = None
                _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
            for epoch in range(start_epoch, n_epochs + 1):
                loss_fn_args['loss_cap'] = loss_cap
                _, _ = train.train_triplet_capped_loss(epoch, train_loader_tripletloss_smote, network, optimizer, verbose=False, cap_calc=cap_calc, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)


learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for loss_cap in loss_caps:
    for i in range(len(learning_rates)): 
        row = ["triplet_loss_capped_smote_average", 2, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], loss_cap, norm]
        rows.append(row)

NameError: name 'loss_fns' is not defined

In [9]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [8]:
# capped smote using triplet loss w/ positive dist squared 

momentum=0
learning_rates = [5e-3, 1e-3, 1e-2]

loss_fn_args = {}


learning_rate_aucs = []

start_epoch = 2

loss_caps = [1, 5, 10]
cap_calc = loss_fns.TripletLossSquaredPositive


for loss_cap in loss_caps:
    for learning_rate in learning_rates:
        aucs = []
        for i in range(5): 
            print(loss_cap)
            model_aucs = []
            network = models.ConvNetWithEmbeddings(2)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(start_epoch):
                loss_fn_args['loss_cap'] = None
                _, _ = train.train_sigmoid_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
            for epoch in range(start_epoch, n_epochs + 1):
                loss_fn_args['loss_cap'] = loss_cap
                _, _ = train.train_triplet_capped_loss(epoch, train_loader_tripletloss_smote, network, optimizer, verbose=False, cap_calc=cap_calc, loss_fn=loss_fns.CappedBCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network, embeddings=True)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)


learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for loss_cap in loss_caps:
    for i in range(len(learning_rates)): 
        row = ["triplet_loss_capped_smote_pos_squared", 2, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], loss_cap, norm]
        rows.append(row)

1

Test set: Avg. loss: 0.0006921598315238952, AUC: 0.5632135


Test set: Avg. loss: 0.0012470570206642151, AUC: 0.7233750000000001


Test set: Avg. loss: 0.0013748981952667236, AUC: 0.7771020000000002


Test set: Avg. loss: 0.0013903254866600037, AUC: 0.807651

1

Test set: Avg. loss: 0.000689566433429718, AUC: 0.6339159999999999


Test set: Avg. loss: 0.001366079866886139, AUC: 0.759145


Test set: Avg. loss: 0.0013456925749778747, AUC: 0.780262


Test set: Avg. loss: 0.001352324604988098, AUC: 0.8117350000000001

1

Test set: Avg. loss: 0.0006955260336399078, AUC: 0.364984


Test set: Avg. loss: 0.0012048510313034057, AUC: 0.7615790000000001


Test set: Avg. loss: 0.0013354987502098084, AUC: 0.806033


Test set: Avg. loss: 0.001402545928955078, AUC: 0.8174680000000001

1

Test set: Avg. loss: 0.0006994863450527191, AUC: 0.3235535


Test set: Avg. loss: 0.001231974244117737, AUC: 0.744665


Test set: Avg. loss: 0.0013687712550163268, AUC: 0.7914589999999999


Test set: Avg. loss: 0.0

10

Test set: Avg. loss: 0.0007007008492946624, AUC: 0.335313


Test set: Avg. loss: 0.0012136767506599425, AUC: 0.744562


Test set: Avg. loss: 0.0012937182188034059, AUC: 0.792217


Test set: Avg. loss: 0.0013339343070983888, AUC: 0.8134770000000001

10

Test set: Avg. loss: 0.0006943095922470093, AUC: 0.4270065


Test set: Avg. loss: 0.0009600128531455994, AUC: 0.73343


Test set: Avg. loss: 0.0012829274535179139, AUC: 0.7855930000000001


Test set: Avg. loss: 0.0013778572082519531, AUC: 0.8141099999999999

10

Test set: Avg. loss: 0.0007034328579902649, AUC: 0.2648275


Test set: Avg. loss: 0.0012563759088516236, AUC: 0.7556390000000001


Test set: Avg. loss: 0.0013986634612083435, AUC: 0.785158


Test set: Avg. loss: 0.0013595693111419678, AUC: 0.808511

10

Test set: Avg. loss: 0.0006898211240768433, AUC: 0.66229


Test set: Avg. loss: 0.0011816049218177796, AUC: 0.6702199999999999


Test set: Avg. loss: 0.0010327804684638977, AUC: 0.723724


Test set: Avg. loss: 0.00106056636571

In [9]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [18]:
# 3 classes

NUM_CLASSES_REDUCED = 3
nums = (0, 3, 1)
ratio = (20, 2, 1)

norm=True

if norm:
    transform=torchvision.transforms.Compose([torchvision.transforms.Normalize(mean=[-0.1390, -0.0688,  0.0125], std=[[1.0098, 0.9996, 0.9935]])])
else:
    transform=None


reduced_train_CIFAR10 = class_sampling.Reduce(train_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True, transform=transform)
reduced_test_CIFAR10 = class_sampling.Reduce(test_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True, transform=transform)

ratio_train_CIFAR10 = class_sampling.Ratio(train_CIFAR10, NUM_CLASSES_REDUCED, ratio, nums=nums, transform=transform)
targets = ratio_train_CIFAR10.labels 
class_count = np.unique(targets, return_counts=True)[1]

smote_train_CIFAR10 = class_sampling.Smote(ratio_train_CIFAR10, 5000 * NUM_CLASSES_REDUCED, transform=transform)


weight = 1. / class_count
samples_weight = weight[targets]
samples_weight = torch.from_numpy(samples_weight)
oversampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(max(class_count) * NUM_CLASSES_REDUCED), replacement=True)
sampler = torch.utils.data.WeightedRandomSampler(samples_weight, len(samples_weight), replacement=True)
undersampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(min(class_count) * NUM_CLASSES_REDUCED), replacement=False)

weight *= max(class_count)

train_loader_reduced = DataLoader(reduced_train_CIFAR10, batch_size=batch_size_train, shuffle=True)  

train_loader_ratio = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, shuffle=True) 

train_loader_oversampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=oversampler)

train_loader_undersampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=undersampler)

train_loader_sampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=sampler)

train_loader_smote = DataLoader(smote_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

test_loader_reduced = DataLoader(reduced_test_CIFAR10, batch_size=batch_size_test, shuffle=True)

In [8]:
# 3 class normal

learning_rates = [1e-4, 1e-3]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_reduced, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["normal", 3, nums, (1, 1, 1), learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)


Test set: Avg. loss: 0.004159178098042806, AUC: 0.5



KeyboardInterrupt: 

In [11]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [15]:
#  3 class ratio

learning_rates = [1e-3]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_ratio, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["ratio", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)


Test set: Avg. loss: 0.0028148902257283527, AUC: 0.5499715000000001


Test set: Avg. loss: 0.0012899534304936728, AUC: 0.5


Test set: Avg. loss: 0.0011812520821889241, AUC: 0.5


Test set: Avg. loss: 0.0011003198623657227, AUC: 0.5


Test set: Avg. loss: 0.0012160149812698365, AUC: 0.496695


Test set: Avg. loss: 0.0012602541049321493, AUC: 0.5


Test set: Avg. loss: 0.0011723772684733072, AUC: 0.5


Test set: Avg. loss: 0.0012640552123387655, AUC: 0.5


Test set: Avg. loss: 0.001084180474281311, AUC: 0.47396475


Test set: Avg. loss: 0.0012727203766504925, AUC: 0.4995


Test set: Avg. loss: 0.0010816088120142618, AUC: 0.5045000000000001


Test set: Avg. loss: 0.0011214701334635417, AUC: 0.5035


Test set: Avg. loss: 0.0026806603272755943, AUC: 0.49925


Test set: Avg. loss: 0.0012301311095555623, AUC: 0.5


Test set: Avg. loss: 0.001151394208272298, AUC: 0.5


Test set: Avg. loss: 0.00107947838306427, AUC: 0.5


Test set: Avg. loss: 0.0020958302021026613, AUC: 0.44825000000000004




In [16]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [17]:
# 3 class oversampled 

learning_rates = [1e-3]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_oversampled, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["oversampled", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)



Test set: Avg. loss: 0.003038533369700114, AUC: 0.5


Test set: Avg. loss: 0.0010433460474014282, AUC: 0.574998


Test set: Avg. loss: 0.0011814462741216024, AUC: 0.7483075


Test set: Avg. loss: 0.001038165827592214, AUC: 0.7227574999999999


Test set: Avg. loss: 0.007914267698923746, AUC: 0.49925


Test set: Avg. loss: 0.0010645556449890136, AUC: 0.37466975


Test set: Avg. loss: 0.001031708836555481, AUC: 0.6914997500000001


Test set: Avg. loss: 0.0011311002572377523, AUC: 0.75428325


Test set: Avg. loss: 0.006719241301218669, AUC: 0.5075000000000001


Test set: Avg. loss: 0.0010695095856984456, AUC: 0.5922592499999999


Test set: Avg. loss: 0.0010078495343526204, AUC: 0.66097125


Test set: Avg. loss: 0.0009656443595886231, AUC: 0.68574


Test set: Avg. loss: 0.002060540755589803, AUC: 0.5615957500000001


Test set: Avg. loss: 0.0010401540199915568, AUC: 0.3627755


Test set: Avg. loss: 0.0009678711692492167, AUC: 0.6725415


Test set: Avg. loss: 0.0009192776083946228, AUC: 0.67

In [18]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)
rows = [] 

In [19]:
# 3 class undersampled  

learning_rates = [1e-3]

learning_rate_aucs = []

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_undersampled, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["undersampled", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)



Test set: Avg. loss: 0.0020289310614267984, AUC: 0.4756965


Test set: Avg. loss: 0.001091553012530009, AUC: 0.48456375


Test set: Avg. loss: 0.0010886570612589517, AUC: 0.4860155


Test set: Avg. loss: 0.0010853137572606406, AUC: 0.5097382500000001


Test set: Avg. loss: 0.0019787512222925823, AUC: 0.57222475


Test set: Avg. loss: 0.0011532610654830932, AUC: 0.49576575000000006


Test set: Avg. loss: 0.001145702044169108, AUC: 0.4937625


Test set: Avg. loss: 0.001140582799911499, AUC: 0.49350675


Test set: Avg. loss: 0.005084774017333984, AUC: 0.5


Test set: Avg. loss: 0.0010802105665206909, AUC: 0.50175


Test set: Avg. loss: 0.0010755322376887005, AUC: 0.502


Test set: Avg. loss: 0.0010662730137507121, AUC: 0.5022505


Test set: Avg. loss: 0.009249690055847169, AUC: 0.49975


Test set: Avg. loss: 0.0010924884875615438, AUC: 0.50599775


Test set: Avg. loss: 0.0010882760683695474, AUC: 0.5044992500000001


Test set: Avg. loss: 0.0010864359935124715, AUC: 0.50449775


Test set:

In [20]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = [] 

In [21]:
#  3 class weighted

learning_rates = [1e-3]

learning_rate_aucs = []

loss_fn_args={}
loss_fn_args['weight'] = torch.from_numpy(weight).float()


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_ratio, network, optimizer, verbose=False, loss_fn_args=loss_fn_args)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["weighted", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)



Test set: Avg. loss: 0.00608756939570109, AUC: 0.501


Test set: Avg. loss: 0.0010771948893864949, AUC: 0.5007499999999999


Test set: Avg. loss: 0.001078349749247233, AUC: 0.49949999999999994


Test set: Avg. loss: 0.0010784133275349936, AUC: 0.50025


Test set: Avg. loss: 0.00372534704208374, AUC: 0.50023525


Test set: Avg. loss: 0.0010370986064275106, AUC: 0.49925


Test set: Avg. loss: 0.0010473136504491171, AUC: 0.49649774999999996


Test set: Avg. loss: 0.0010565840800603231, AUC: 0.5224875


Test set: Avg. loss: 0.0020539817810058592, AUC: 0.50625


Test set: Avg. loss: 0.0010878965854644776, AUC: 0.50025


Test set: Avg. loss: 0.001087069312731425, AUC: 0.49999999999999994


Test set: Avg. loss: 0.0010845698912938435, AUC: 0.50025


Test set: Avg. loss: 0.006659661134084066, AUC: 0.49401975000000004


Test set: Avg. loss: 0.0010928993225097657, AUC: 0.53216375


Test set: Avg. loss: 0.0010782272815704345, AUC: 0.54923675


Test set: Avg. loss: 0.0010686718225479126, AUC: 0.55

In [22]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = [] 

In [13]:
#  3 class focal loss

learning_rates = [1e-3]

learning_rate_aucs = []

loss_fn_args={}
loss_fn_args['reduction'] = 'mean'


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax(epoch, train_loader_ratio, network, optimizer, verbose=False, loss_fn=loss_fns.SoftmaxFocalLoss, loss_fn_args=loss_fn_args)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["focal_loss", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)


AttributeError: 'bool' object has no attribute 'any'

In [24]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [19]:
#  3 class SMOTE

learning_rates = [1e-4, 5e-4, 1e-5, 1e-3]

learning_rate_aucs = []

loss_fn_args = {}
loss_fn_args['loss_cap'] = None

for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_softmax_with_smote(epoch, train_loader_smote, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["smote", 3, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3], None, norm]
    rows.append(row)


RuntimeError: The size of tensor a (32) must match the size of tensor b (3) at non-singleton dimension 2

In [15]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [9]:
# 3 class capped smote 


momentum=0
learning_rates = [1e-3, 5e-4, 1e-4]


cap_aucs = []

caps = [1, 5, 10]

for cap in caps:
    
    loss_fn_args = {}
    loss_fn_args['loss_cap'] = cap
    
    learning_rate_aucs = []

    for learning_rate in learning_rates:
        aucs = []
        for i in range(10):
            model_aucs = []
            network = models.ConvNet(NUM_CLASSES_REDUCED)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_softmax(test_loader_reduced, network) 
            model_aucs.append(auc)
            for epoch in range(n_epochs):
                _, _ = train.train_softmax_with_smote(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_softmax(test_loader_reduced, network)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)

    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    
    
    cap_aucs.append([auc_mean, auc_variance])

    
    
for i in range(len(cap_aucs)):
    auc_mean = cap_aucs[i][0]
    auc_variance = cap_aucs[i][1]
    cap = caps[i]
    for i in range(len(learning_rates)): 
        row = ["capped_smote", NUM_CLASSES_REDUCED, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm]
        rows.append(row)

print(rows)



Test set: Avg. loss: 0.004159178098042806, AUC: 0.5


Test set: Avg. loss: 0.0010975551207860312, AUC: 0.6297499999999999


Test set: Avg. loss: 0.0006656141678492228, AUC: 0.7525


Test set: Avg. loss: 0.0010653618971506755, AUC: 0.6115


Test set: Avg. loss: 0.001217594623565674, AUC: 0.5495857500000001


Test set: Avg. loss: 0.0011367549101511637, AUC: 0.6367499999999999


Test set: Avg. loss: 0.0010483775536219279, AUC: 0.6514999999999999


Test set: Avg. loss: 0.0007630239725112915, AUC: 0.7260000000000001


Test set: Avg. loss: 0.0012967588504155477, AUC: 0.44993225000000003


Test set: Avg. loss: 0.0010262293815612793, AUC: 0.6295


Test set: Avg. loss: 0.0008588801821072897, AUC: 0.683


Test set: Avg. loss: 0.0006734636823336283, AUC: 0.78375


Test set: Avg. loss: 0.009516664822896322, AUC: 0.4979995


Test set: Avg. loss: 0.00063530965646108, AUC: 0.69475


Test set: Avg. loss: 0.000929928461710612, AUC: 0.6829999999999999


Test set: Avg. loss: 0.0007287528316179911, AUC: 


Test set: Avg. loss: 0.0010305375258127848, AUC: 0.6913247499999999


Test set: Avg. loss: 0.0010350863933563232, AUC: 0.6280855000000001


Test set: Avg. loss: 0.0010418128967285157, AUC: 0.5992064999999999


Test set: Avg. loss: 0.03673971430460612, AUC: 0.5


Test set: Avg. loss: 0.0010967044830322266, AUC: 0.5


Test set: Avg. loss: 0.0010927757024765015, AUC: 0.4995


Test set: Avg. loss: 0.00109928297996521, AUC: 0.5


Test set: Avg. loss: 0.009726393699645995, AUC: 0.48325


Test set: Avg. loss: 0.0010913198391596477, AUC: 0.56702925


Test set: Avg. loss: 0.0010592830578486125, AUC: 0.5896625


Test set: Avg. loss: 0.00109747048219045, AUC: 0.541984


Test set: Avg. loss: 0.013137797037760416, AUC: 0.49675


Test set: Avg. loss: 0.0010657407840092977, AUC: 0.7087220000000001


Test set: Avg. loss: 0.0011155157089233398, AUC: 0.5314895000000001


Test set: Avg. loss: 0.0011556921402613321, AUC: 0.5110129999999999


Test set: Avg. loss: 0.004617925008138021, AUC: 0.5070490000000


Test set: Avg. loss: 0.0011017917394638061, AUC: 0.5


Test set: Avg. loss: 0.0011000678539276124, AUC: 0.5


Test set: Avg. loss: 0.0010989853541056316, AUC: 0.5


Test set: Avg. loss: 0.012646420160929362, AUC: 0.47450000000000003


Test set: Avg. loss: 0.0010960377852121989, AUC: 0.6284795


Test set: Avg. loss: 0.0010728757381439208, AUC: 0.5803695


Test set: Avg. loss: 0.0011331966718037922, AUC: 0.5152655


Test set: Avg. loss: 0.0023012900352478026, AUC: 0.50575


Test set: Avg. loss: 0.001067526618639628, AUC: 0.537042


Test set: Avg. loss: 0.001066692630449931, AUC: 0.539551


Test set: Avg. loss: 0.0011165502866109212, AUC: 0.51325075


Test set: Avg. loss: 0.005958204746246338, AUC: 0.5135000000000001


Test set: Avg. loss: 0.0010548818508783976, AUC: 0.610457


Test set: Avg. loss: 0.001052526871363322, AUC: 0.5903295000000001


Test set: Avg. loss: 0.001153125564257304, AUC: 0.5192565


Test set: Avg. loss: 0.004796805699666341, AUC: 0.52476475


Test set: Avg. loss: 0.

In [10]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [12]:
# 3 class euclidean distance capped smote 


momentum=0
learning_rates = [5e-5, 5e-4, 1e-4]


cap_aucs = []

caps = [1, 5, 10]

loss_fn_args = {}
loss_fn_args['distance'] = 'euclidean'


for cap in caps:
    
    loss_fn_args['loss_cap'] = cap
    
    learning_rate_aucs = []

    for learning_rate in learning_rates:
        aucs = []
        for i in range(10):
            model_aucs = []
            network = models.ConvNetWithEmbeddings(NUM_CLASSES_REDUCED)
            optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
            _, auc = metric_utils.auc_softmax(test_loader_reduced, network, embeddings=True) 
            model_aucs.append(auc)
            for epoch in range(n_epochs):
                _, _ = train.train_softmax_with_embeddings(epoch, train_loader_smote, network, optimizer, verbose=False, loss_fn=loss_fns.CappedCELoss, loss_fn_args=loss_fn_args)
                if (epoch + 1) % 10 == 0: 
                    _, auc = metric_utils.auc_softmax(test_loader_reduced, network, embeddings=True)
                    model_aucs.append(auc)
            aucs.append(model_aucs)
        learning_rate_aucs.append(aucs)

    learning_rate_aucs = np.asarray(learning_rate_aucs)

    auc_mean = np.mean(learning_rate_aucs, axis=1)
    auc_variance = np.var(learning_rate_aucs, axis=1)
    
    
    cap_aucs.append([auc_mean, auc_variance])

    
    
for i in range(len(cap_aucs)):
    auc_mean = cap_aucs[i][0]
    auc_variance = cap_aucs[i][1]
    cap = caps[i]
    for i in range(len(learning_rates)): 
        row = ["distance_capped_smote", NUM_CLASSES_REDUCED, nums, ratio, learning_rates[i],
                auc_mean[i][0], auc_variance[i][0], 
                auc_mean[i][1], auc_variance[i][1],
                auc_mean[i][2], auc_variance[i][2],
                auc_mean[i][3], auc_variance[i][3], cap, norm]
        rows.append(row)

print(rows)



Test set: Avg. loss: 0.0078015821774800615, AUC: 0.49625


Test set: Avg. loss: 0.001081034223238627, AUC: 0.51192


Test set: Avg. loss: 0.0010818347930908203, AUC: 0.5092574999999999


Test set: Avg. loss: 0.0010877530177434286, AUC: 0.59471175


Test set: Avg. loss: 0.006712500254313151, AUC: 0.5


Test set: Avg. loss: 0.0011000588337580362, AUC: 0.5037455


Test set: Avg. loss: 0.0011020750999450684, AUC: 0.5004995


Test set: Avg. loss: 0.001104021390279134, AUC: 0.4999995


Test set: Avg. loss: 0.001641412854194641, AUC: 0.5240275


Test set: Avg. loss: 0.0011246995925903321, AUC: 0.564403


Test set: Avg. loss: 0.0011237755616505941, AUC: 0.5504524999999999


Test set: Avg. loss: 0.0011289521853129069, AUC: 0.58037975


Test set: Avg. loss: 0.004778544743855795, AUC: 0.5345000000000001


Test set: Avg. loss: 0.0011649170319239298, AUC: 0.5652305


Test set: Avg. loss: 0.001163023034731547, AUC: 0.59888525


Test set: Avg. loss: 0.0011398308277130126, AUC: 0.598623


Test set: A


Test set: Avg. loss: 0.001166247844696045, AUC: 0.48674625000000005


Test set: Avg. loss: 0.001174075166384379, AUC: 0.48724100000000004


Test set: Avg. loss: 0.0011709243059158324, AUC: 0.490257


Test set: Avg. loss: 0.009658121744791666, AUC: 0.5005000000000001


Test set: Avg. loss: 0.0011268941561381023, AUC: 0.47115225000000005


Test set: Avg. loss: 0.0011330329577128092, AUC: 0.4739225


Test set: Avg. loss: 0.0012209938367207845, AUC: 0.579674


Test set: Avg. loss: 0.0010735984245936075, AUC: 0.52853025


Test set: Avg. loss: 0.0011176213026046753, AUC: 0.53512575


Test set: Avg. loss: 0.0011161125103632608, AUC: 0.5316354999999999


Test set: Avg. loss: 0.001127867062886556, AUC: 0.549112


Test set: Avg. loss: 0.003836607058842977, AUC: 0.50138575


Test set: Avg. loss: 0.0010821324189503989, AUC: 0.5089839999999999


Test set: Avg. loss: 0.0010832757155100504, AUC: 0.51515325


Test set: Avg. loss: 0.001098487655321757, AUC: 0.550893


Test set: Avg. loss: 0.0009752304


Test set: Avg. loss: 0.0011330342292785644, AUC: 0.5235


Test set: Avg. loss: 0.0011348271369934083, AUC: 0.5427500000000001


Test set: Avg. loss: 0.001139047622680664, AUC: 0.5818490000000001


Test set: Avg. loss: 0.002080832600593567, AUC: 0.48411425


Test set: Avg. loss: 0.0011403302748998006, AUC: 0.49874950000000007


Test set: Avg. loss: 0.0011391813755035401, AUC: 0.49874975000000005


Test set: Avg. loss: 0.001156262993812561, AUC: 0.495748


Test set: Avg. loss: 0.001705811619758606, AUC: 0.5480827500000001


Test set: Avg. loss: 0.0011754041910171508, AUC: 0.532629


Test set: Avg. loss: 0.0012903087536493938, AUC: 0.53393825


Test set: Avg. loss: 0.00134560227394104, AUC: 0.52968825


Test set: Avg. loss: 0.005548909982045491, AUC: 0.4260195


Test set: Avg. loss: 0.0011066407362620035, AUC: 0.6257185


Test set: Avg. loss: 0.001109532912572225, AUC: 0.6433625000000001


Test set: Avg. loss: 0.0011481906175613403, AUC: 0.7056715000000001


Test set: Avg. loss: 0.002029

In [13]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []